In [1]:
#### kusto query public server

import pandas as pd
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.kusto.data.exceptions import KustoServiceError
import traceback
import logging
import time

class KustoQuery(object):
    def __init__(self, **kwargs):        
        # self.client = None
        # self.kcsb = None
        # initialize client
        self.get_kusto_client()
        
    def get_kusto_client(self, **kwargs):
        import urllib3
        urllib3.disable_warnings()
    
        # self.cluster = kwargs.get('cluster', 'https://deepicm.westus2.kusto.windows.net')
        # self.cluster = 'https://msraaiops.eastus.kusto.windows.net'
        KustoCluster = 'https://infraoptimization.westus2.kusto.windows.net' #'https://msraaiops.eastus.kusto.windows.net'
        
        self.db = kwargs.get('database', 'Shared')#'WorkloadProfiling')
        self.db = 'workload'
        # self.application_name = kwargs.get('application_name', "67862165-cbdc-4811-bb83-d6d945a98b6b")
        # self.AAD_tenant_id = kwargs.get('AAD_tenant_id', '72f988bf-86f1-41af-91ab-2d7cd011db47')     
   
        # self.application_name = kwargs.get('application_name', "549d2a15-4a98-4fee-becb-773e531b0f7c")
        # self.AAD_tenant_id = kwargs.get('AAD_tenant_id', '72f988bf-86f1-41af-91ab-2d7cd011db47')

        # ClientId = self.application_name
        # ClientSecret = self.application_pwd
        # AADTenantId = self.AAD_tenant_id
        
        # KustoCluster = self.cluster
        #kcsb = KustoConnectionStringBuilder.with_aad_device_authentication(KustoCluster) ### use this one to access via AAD
        kcsb = KustoConnectionStringBuilder.with_az_cli_authentication(KustoCluster)
        #kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(KustoCluster, ClientId, ClientSecret, AADTenantId)
        kusto_client = KustoClient(kcsb)
        kusto_client._session.verify = False
        self.kcsb = kcsb
        self.client = kusto_client 
        
    def query(self, kql=None, return_type='dataframe'):
        if self.client is None:
            self.get_kusto_client()
        try:
            response = self.client.execute(self.db, kql)
            res = response.primary_results[0]
            if return_type == 'dataframe':
                return dataframe_from_result_table(res)
        except KustoServiceError as error:
            print("Error:",error)
            print('Retrying after sleeping for 10 sec..')
            #time.sleep(600)
            time.sleep(10)
            
            response = self.client.execute(self.db, kql)
            res = response.primary_results[0]
            if return_type == 'dataframe':
                return dataframe_from_result_table(res)
            else:
                return None
            # print("KustoServiceError: ", error)
            #logging.getLogger().warn(f'exception error = {error}, kql = {kql}')
            #logging.getLogger().warn(traceback.format_exc())
            #return None

In [2]:
query = """.set-or-append LLMApiRequestTracingEvent_Global <|
cluster('modeldqasprodeus2.eastus').database('modeldqaseus2db').LLMApiRequestTracingEvent_Global
| where TIMESTAMP between (datetime({start}) .. datetime({end}))"""
kq = KustoQuery()
from datetime import datetime
# datetime_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
# print(datetime_string)
startTime = datetime.fromisoformat('2024-06-26T00:00:00Z')
for i in range(0, 24*4, 6):
    st = startTime + pd.DateOffset(hours=i)
    et = startTime + pd.DateOffset(hours=i+6)
    q = query.format(start=st.strftime("%Y-%m-%dT%H:%M:%SZ"), end=et.strftime("%Y-%m-%dT%H:%M:%SZ"))
    print(q)
    # time = datetime.now()
    # df = kq.query(q)
    # time = datetime.now() - time
    # print(st, et, time)

.set-or-append LLMApiRequestTracingEvent_Global <|
cluster('modeldqasprodeus2.eastus').database('modeldqaseus2db').LLMApiRequestTracingEvent_Global
| where TIMESTAMP between (datetime(2024-06-26T00:00:00Z) .. datetime(2024-06-26T06:00:00Z))
.set-or-append LLMApiRequestTracingEvent_Global <|
cluster('modeldqasprodeus2.eastus').database('modeldqaseus2db').LLMApiRequestTracingEvent_Global
| where TIMESTAMP between (datetime(2024-06-26T06:00:00Z) .. datetime(2024-06-26T12:00:00Z))
.set-or-append LLMApiRequestTracingEvent_Global <|
cluster('modeldqasprodeus2.eastus').database('modeldqaseus2db').LLMApiRequestTracingEvent_Global
| where TIMESTAMP between (datetime(2024-06-26T12:00:00Z) .. datetime(2024-06-26T18:00:00Z))
.set-or-append LLMApiRequestTracingEvent_Global <|
cluster('modeldqasprodeus2.eastus').database('modeldqaseus2db').LLMApiRequestTracingEvent_Global
| where TIMESTAMP between (datetime(2024-06-26T18:00:00Z) .. datetime(2024-06-27T00:00:00Z))
.set-or-append LLMApiRequestTracingE